<h1> 2d. Distributed training and monitoring </h1>

In this notebook, we refactor to call ```train_and_evaluate``` instead of hand-coding our ML pipeline. This allows us to carry out evaluation as part of our training loop instead of as a separate step. It also adds in failure-handling that is necessary for distributed training capabilities.

We also use TensorBoard to monitor the training.

In [3]:
import tensorflow as tf
print tf.__version__

1.4.1


In [4]:
import datalab.bigquery as bq
import tensorflow as tf
import numpy as np
import shutil

<h2> Input </h2>

Read data created in Lab1a, but this time make it more general, so that we are reading in batches.  Instead of using Pandas, we will use add a filename queue to the TensorFlow graph.

In [5]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, mode, batch_size=512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults=DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label
    
    dataset = tf.data.TextLineDataset(filename).map(decode_csv)
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size=10*batch_size)
    else:
        num_epochs = 1 # end-of-input after this
 
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  
  return _input_fn

<h2> Create features out of input data </h2>

For now, pass these through.  (same as previous lab)

In [6]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers'),
]

def add_more_features(feats):
  # nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> train_and_evaluate </h2>

In [7]:
def serving_input_fn():
    feature_placeholders = {
      'pickuplon' : tf.placeholder(tf.float32, [None]),
      'pickuplat' : tf.placeholder(tf.float32, [None]),
      'dropofflat' : tf.placeholder(tf.float32, [None]),
      'dropofflon' : tf.placeholder(tf.float32, [None]),
      'passengers' : tf.placeholder(tf.float32, [None]),
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.LinearRegressor(
                         model_dir=output_dir,
                         feature_columns=feature_cols)
    train_spec=tf.estimator.TrainSpec(
                         input_fn=read_dataset('./taxi-train.csv', mode=tf.contrib.learn.ModeKeys.TRAIN),
                         max_steps=num_train_steps)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec=tf.estimator.EvalSpec(
                         input_fn=read_dataset('./taxi-valid.csv', mode=tf.contrib.learn.ModeKeys.EVAL),
                         steps=None,
                         start_delay_secs=1, # start evaluating after N seconds
                         throttle_secs=10,  # evaluate every N seconds
                         exporters=exporter)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


# run training    
OUTDIR='taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors=True) # start fresh each time
train_and_evaluate(OUTDIR, 5000)

INFO:tensorflow:Using default config.
[2018-01-28 07:02:26,584] {tf_logging.py:82} INFO - Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f50bc2c1410>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'taxi_trained', '_save_summary_steps': 100}
[2018-01-28 07:02:26,587] {tf_logging.py:82} INFO - Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f50bc2c1410>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hour

<h2> Monitoring with TensorBoard </h2>

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('./taxi_trained')
TensorBoard().list()

In [ ]:
# to stop TensorBoard
TensorBoard().stop(9049)
print 'stopped TensorBoard'

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License